In [25]:
import pandas as pd

train_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/train_set_fixed.csv"
test_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/test_set_fixed.csv"

# Reload datasets
train_df = pd.read_csv(train_file, encoding="utf-8")
test_df = pd.read_csv(test_file, encoding="utf-8")

print("Train Dataset Loaded! Shape:", train_df.shape)
print("Test Dataset Loaded! Shape:", test_df.shape)

Train Dataset Loaded! Shape: (1618172, 16)
Test Dataset Loaded! Shape: (404543, 16)


In [27]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Prepare data (Remove label column for training, since Isolation Forest is unsupervised)
features = [col for col in train_df.columns if col != "Label"]

# Standardize the feature values for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
X_test = scaler.transform(test_df[features])  # Use the same scaler for test set

# Train Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination="auto", random_state=42)
iso_forest.fit(X_train)

# Predict anomalies (-1 = anomaly, 1 = normal)
test_predictions = iso_forest.predict(X_test)

# Convert to binary format (to match ground truth: "Benign" → 1, "Attack" → -1)
y_test = np.where(test_df["Label"] == "Benign", 1, -1)

# Compare predictions vs. actual labels
conf_matrix = confusion_matrix(y_test, test_predictions)
class_report = classification_report(y_test, test_predictions, target_names=["Attack (-1)", "Benign (1)"])

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[  4524 133775]
 [ 36505 229739]]

Classification Report:
               precision    recall  f1-score   support

 Attack (-1)       0.11      0.03      0.05    138299
  Benign (1)       0.63      0.86      0.73    266244

    accuracy                           0.58    404543
   macro avg       0.37      0.45      0.39    404543
weighted avg       0.45      0.58      0.50    404543



In [29]:
# Check class distribution in train and test datasets
train_class_distribution = train_df["Label"].value_counts(normalize=True) * 100  # Percentage distribution
test_class_distribution = test_df["Label"].value_counts(normalize=True) * 100

# Display results
print("Train Set Distribution:\n", train_class_distribution)
print("\nTest Set Distribution:\n", test_class_distribution)

Train Set Distribution:
 Label
Benign                    65.813461
DDoS attacks-LOIC-HTTP    28.485229
Infilteration              3.117963
DoS attacks-GoldenEye      2.049473
DoS attacks-Slowloris      0.508475
Brute Force -Web           0.017921
Brute Force -XSS           0.007478
Name: proportion, dtype: float64

Test Set Distribution:
 Label
Benign                    65.813523
DDoS attacks-LOIC-HTTP    28.485229
Infilteration              3.118086
DoS attacks-GoldenEye      2.049473
DoS attacks-Slowloris      0.508475
Brute Force -Web           0.017798
Brute Force -XSS           0.007416
Name: proportion, dtype: float64


In [33]:
# Check statistical summary of features
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Bwd Pkt Len Mean,1618172.0,1.308282e+02,1.763963e+02,0.000000,0.000000,7.700000e+01,2.258571e+02,1.827135e+03
Flow IAT Mean,1618172.0,8.449768e+06,2.019820e+07,0.333333,7784.666667,1.908531e+05,3.003813e+06,1.199900e+08
Fwd Pkt Len Mean,1618172.0,3.739131e+01,5.659224e+01,0.000000,0.000000,1.033333e+01,5.100000e+01,2.946679e+03
Flow IAT Std,1618172.0,1.096791e+06,4.120390e+06,0.000000,0.000000,5.781879e+03,4.681014e+05,8.418912e+07
TotLen Fwd Pkts,1618172.0,1.930431e+03,1.433502e+05,0.000000,0.000000,3.100000e+01,2.860000e+02,1.443918e+08
Flow Duration,1618172.0,1.745886e+07,3.302318e+07,1.000000,14093.000000,1.412144e+06,1.441877e+07,1.200000e+08
Bwd Pkts/s,1618172.0,3.864629e+03,3.747911e+04,0.000000,0.000000,2.438074e+00,2.645730e+01,2.000000e+06
Flow Pkts/s,1618172.0,2.389273e+04,1.865377e+05,0.016668,0.402143,5.791174e+00,1.719986e+02,4.000000e+06
Fwd IAT Std,1618172.0,1.000956e+06,3.848971e+06,0.000000,0.000000,9.192388e+00,4.570444e+04,8.462898e+07
Flow Byts/s,1618172.0,1.680158e+05,3.198542e+06,0.000000,0.000000,6.140187e+02,2.399163e+03,1.230000e+09


In [35]:
# Add predictions to the test dataset
test_df["Predicted_Label"] = test_predictions  # Assuming you stored Isolation Forest's output in 'test_predictions'

# Count misclassifications
misclassified = test_df[test_df["Label"] != test_df["Predicted_Label"]]

# Check which attack types were misclassified the most
print(misclassified["Label"].value_counts())

Label
Benign                    266244
DDoS attacks-LOIC-HTTP    115235
Infilteration              12614
DoS attacks-GoldenEye       8291
DoS attacks-Slowloris       2057
Brute Force -Web              72
Brute Force -XSS              30
Name: count, dtype: int64


In [11]:
import os

dataset_path = "/Users/akashthanneeru/Desktop/INADS_Data/Data/"
train_file = os.path.join(dataset_path, "train_set_fixed.csv")
test_file = os.path.join(dataset_path, "test_set_fixed.csv")

# Check if files exist and try reading the first line
for file in [train_file, test_file]:
    if os.path.exists(file):
        print(f" {os.path.basename(file)} exists! Checking readability...")
        with open(file, "r", encoding="utf-8") as f:
            print(f.readline())  # Print first line to verify format
    else:
        print(f" {os.path.basename(file)} does not exist!")

 train_set_fixed.csv exists! Checking readability...
Bwd Pkt Len Mean,Flow IAT Mean,Fwd Pkt Len Mean,Flow IAT Std,TotLen Fwd Pkts,Flow Duration,Bwd Pkts/s,Flow Pkts/s,Fwd IAT Std,Flow Byts/s,Fwd Pkt Len Max,Flow IAT Max,Init Fwd Win Byts,Fwd Seg Size Min,Dst Port,Label

 test_set_fixed.csv exists! Checking readability...
Bwd Pkt Len Mean,Flow IAT Mean,Fwd Pkt Len Mean,Flow IAT Std,TotLen Fwd Pkts,Flow Duration,Bwd Pkts/s,Flow Pkts/s,Fwd IAT Std,Flow Byts/s,Fwd Pkt Len Max,Flow IAT Max,Init Fwd Win Byts,Fwd Seg Size Min,Dst Port,Label



In [13]:
import os

dataset_path = "/Users/akashthanneeru/Desktop/INADS_Data/Data/"
train_file = os.path.abspath(os.path.join(dataset_path, "train_set_fixed.csv"))
test_file = os.path.abspath(os.path.join(dataset_path, "test_set_fixed.csv"))

print("Train Dataset Path:", train_file)
print("Test Dataset Path:", test_file)

print("Train File Exists:", os.path.exists(train_file))
print("Test File Exists:", os.path.exists(test_file))

Train Dataset Path: /Users/akashthanneeru/Desktop/INADS_Data/Data/train_set_fixed.csv
Test Dataset Path: /Users/akashthanneeru/Desktop/INADS_Data/Data/test_set_fixed.csv
Train File Exists: True
Test File Exists: True


In [15]:
with open("/Users/akashthanneeru/Desktop/INADS_Data/Data/train_set_fixed.csv", "r", encoding="utf-8") as f:
    print(f.readline())  # Print first line to confirm readability

with open("/Users/akashthanneeru/Desktop/INADS_Data/Data/test_set_fixed.csv", "r", encoding="utf-8") as f:
    print(f.readline())  # Print first line to confirm readability

Bwd Pkt Len Mean,Flow IAT Mean,Fwd Pkt Len Mean,Flow IAT Std,TotLen Fwd Pkts,Flow Duration,Bwd Pkts/s,Flow Pkts/s,Fwd IAT Std,Flow Byts/s,Fwd Pkt Len Max,Flow IAT Max,Init Fwd Win Byts,Fwd Seg Size Min,Dst Port,Label

Bwd Pkt Len Mean,Flow IAT Mean,Fwd Pkt Len Mean,Flow IAT Std,TotLen Fwd Pkts,Flow Duration,Bwd Pkts/s,Flow Pkts/s,Fwd IAT Std,Flow Byts/s,Fwd Pkt Len Max,Flow IAT Max,Init Fwd Win Byts,Fwd Seg Size Min,Dst Port,Label



In [19]:
import os

train_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/train_set_fixed.csv"
test_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/test_set_fixed.csv"

# Check file permissions
print("Train File Permissions:", oct(os.stat(train_file).st_mode)[-3:])
print("Test File Permissions:", oct(os.stat(test_file).st_mode)[-3:])

Train File Permissions: 644
Test File Permissions: 644


In [21]:
import os

dataset_path = "/Users/akashthanneeru/Desktop/INADS_Data/Data/"
print("Jupyter Directory Listing:")
print(os.listdir(dataset_path))

Jupyter Directory Listing:
['Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv', 'DoS_Attacks_Filtered.csv', 'Merged-Dataset-Final.csv', 'feature_extraction_final.csv', 'test_set_fixed.csv', 'Benign_Traffic.csv', 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv', 'train_set_fixed.csv', 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv']


In [23]:
import pandas as pd

train_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/train_set_fixed.csv"
test_file = "/Users/akashthanneeru/Desktop/INADS_Data/Data/test_set_fixed.csv"

# Force Pandas to read by explicitly opening the file
with open(train_file, "r", encoding="utf-8") as f:
    train_df = pd.read_csv(f)
    
with open(test_file, "r", encoding="utf-8") as f:
    test_df = pd.read_csv(f)

print("Train Dataset Shape:", train_df.shape)
print("Test Dataset Shape:", test_df.shape)

Train Dataset Shape: (1618172, 16)
Test Dataset Shape: (404543, 16)
